# Creating a EfficientDet model using B0 backbone

We first install the EfficientDet library

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
!git clone https://github.com/joheras/EfficientDet

In [0]:
cd EfficientDet/

In [0]:
!pip install -r requirements.txt

We load the necessary code and start the training process.

We download the dataset. If you are working with Google Colab, you have several options to download the dataset in this notebook, see the available options in the LabelDetection documentation.

In [0]:
!unzip datasets.zip

We train the model.



In [0]:
!python3 train.py --snapshot imagenet --phi 0 --gpu 0 --random-transform --compute-val-loss --freeze-backbone --batch-size 4 --epochs 25 --steps 32 pascalCustom datasets/VOCdataset/

In [0]:
!python3 train.py --snapshot checkpoints/*/pascalCustom_25_*.h5 --phi 0 --gpu 0 --random-transform --compute-val-loss --freeze-bn --batch-size 4 --epochs 30 --steps 32 pascalCustom datasets/VOCdataset/

The model was evaluated at the end of the previous execution.

In [0]:
!mv checkpoints/*/pascalCustom_30_*.h5 output.h5

In [0]:
!rm -rf checkpoints/*

At the end you will have a file called model.h5 that together with the names file can be included in the application to be employed with new images. 

----------------------

# Data distillation

We first install the library for ensemble methods.

In [0]:
!git clone  https://github.com/ancasag/ensembleObjectDetection.git

In [0]:
cd ensembleObjectDetection/TestTimeAugmentation

We install additional libraries that are required.

In [0]:
!pip install clodsa
!pip install mrcnn
!pip install keras-retinanet
!pip install gluoncv
!pip install mxnet

The following cells apply data distillation.

In [0]:
import testTimeAugmentation
import function
import os
import shutil
import argparse
import ensembleOptions
from mainTTA import tta
from imutils import paths

In [0]:
pathImg = '/content/EfficientDet/datasets/unlabelled/'

In [0]:
effimodel = testTimeAugmentation.Efficient('/content/EfficientDet/output.h5', '/content/EfficientDet/datasets/VOCdataset/classes.csv')

In [0]:
myTechniques = [ "histo","hflip","none"]

In [0]:
option = "affirmative"

In [0]:
tta(effimodel,myTechniques,pathImg,option)

Now, we reorganize the dataset.

In [0]:
!ls /content/EfficientDet/datasets/unlabelled/*.jpg >> /content/EfficientDet/datasets/VOCdataset/ImageSets/Main/train.txt

In [0]:
!sed -i 's#/content/EfficientDet/datasets/unlabelled/##g' /content/EfficientDet/datasets/VOCdataset/ImageSets/Main/train.txt

In [0]:
!sed -i 's#.jpg##g' /content/EfficientDet/datasets/VOCdataset/ImageSets/Main/train.txt

In [0]:
!mv /content/EfficientDet/datasets/unlabelled/*.jpg /content/EfficientDet/datasets/VOCdataset/JPEGImages/

In [0]:
!mv /content/EfficientDet/datasets/unlabelled/*.xml /content/EfficientDet/datasets/VOCdataset/Annotations/

In [0]:
cd /content/EfficientDet

Finally, we retrain the model and evaluate its performance. 


In [0]:
!python3 train.py --snapshot imagenet --phi 0 --gpu 0 --random-transform --compute-val-loss --freeze-backbone --batch-size 4 --epochs 25 --steps 32 pascalCustom datasets/VOCdataset/

In [0]:
!python3 train.py --snapshot checkpoints/*/pascalCustom_25_*.h5 --phi 0 --gpu 0 --random-transform --compute-val-loss --freeze-bn  --batch-size 4 --epochs 25 --steps 32 pascalCustom datasets/VOCdataset/

In [0]:
!mv checkpoints/*/pascalCustom_30_*.h5 output.h5

---------------------------------

# Using the model in LabelDetection

If you want to use the trained model with LabelDetection, you must download the following files:

- EfficientDet/output.h5
- EfficientDet/datasets/classes.names